 ### 1. Changing string representation
 * To change the string representation of an instance, define the **__str__()** and ** __repr__()** methods.

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    # The format code {0.x} specifies the x-attribute of argument 0.
    # So, in the following function, the 0 is actually the instance self
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)

    def __str__(self):
        return '({0.x!s}, {0.y!s})'.format(self)


In [2]:
p = Pair(3,4)
p   # __repr__() output


Pair(3, 4)

In [3]:
print(p)    # __str__() output


(3, 4)


In [4]:
# the special !r formatting code indicates that the output of __repr__()
# should be used instead of __str__(), the default
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))


p is Pair(3, 4)
p is (3, 4)
